<head>
<b><center>CIENCIA DE DATOS</center>

<center>TALLER 3</center>
<center>2024-1</center>

<center>Profesor: Gabriel Jara </center><br>
<center>Integrantes: Fernando Salgado, Cristián González</center><br>
</head>

Intro

Carga Datos

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score


exitosas = pd.read_csv('data/transacciones_exitosas.csv',low_memory=False)
fallidas = pd.read_csv('data/transacciones_fallidas.csv')
evaluacion = pd.read_csv('data/transacciones_evaluación.csv')

# Eliminar fila con mal formato
exitosas = exitosas.drop(148541)


# Añadir etiquetas a los datos
exitosas['label'] = 0
fallidas['label'] = 1

# Combinar datos exitosos y fallidos
datos = pd.concat([exitosas, fallidas], ignore_index=True)


In [2]:
from sklearn.model_selection import StratifiedKFold

# Separar características y etiquetas
X = datos.drop(columns=['label'])
y = datos['label']

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [3]:
# Separar características y etiquetas
X = datos.drop(columns=['label'])
y = datos['label']

# Normalizar datos
scaler = StandardScaler()
X = scaler.fit_transform(X)

c:\Users\borrar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\extmath.py:1140: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\borrar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\extmath.py:1145: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\borrar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\extmath.py:1165: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [4]:
# Definir modelos y parámetros
models = {
    # 'LogisticRegression': LogisticRegression(),
    # 'SVC': SVC(),
    'RandomForest': RandomForestClassifier()
}

params = {
    # 'LogisticRegression': {'C': [0.1, 1, 10]},
    # 'SVC': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']},
    'RandomForest': {'n_estimators': [100, 200], 'max_depth': [10, 20]}
}

# GridSearchCV con StratifiedKFold para selección de modelos
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
best_models = {}
for model_name in models:
    grid = GridSearchCV(models[model_name], params[model_name], cv=skf, scoring='f1')
    grid.fit(X, y)
    best_models[model_name] = grid.best_estimator_

# Evaluar modelos usando F1-score
for model_name in best_models:
    y_pred = cross_val_predict(best_models[model_name], X, y, cv=skf)
    f1 = f1_score(y, y_pred)
    print(f'{model_name}: F1 Score = {f1}')

RandomForest: F1 Score = 1.0


In [5]:
# Combinar datos de entrenamiento y prueba
X_combined = np.vstack((X_train, X_test))
y_combined = np.concatenate((y_train, y_test))

# Entrenar el mejor modelo con todos los datos disponibles
final_model = best_models['RandomForest']
final_model.fit(X_combined, y_combined)


RandomForestClassifier(max_depth=10)

Predicción y Selección de Transacciones Fallidas

In [ ]:
X_evaluacion = evaluacion.drop(columns=['label'])  # Adjust the columns as necessary
X_evaluacion = scaler.transform(X_evaluacion)

# Predict probabilities of failure
predicciones = final_model.predict_proba(X_evaluacion)[:, 1]

# Select the transactions with the highest probability of failure
evaluacion['probabilidad_falla'] = predicciones
top_200 = evaluacion.nlargest(200, 'probabilidad_falla')

# Save the identifiers of the selected transactions
top_200.to_csv('data/top_200_transacciones_fallidas.csv', index=False)